In [1]:
import os
import tensorflow as tf
from tensorflow import keras
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt 


tf.random.set_seed(22)
np.random.seed(22)

# 保存图片， 将 100 张图片放到一张图片上显示
def save_images(imgs, name):
    new_im = Image.new('L', (280, 280))

    index = 0
    for i in range(0, 280, 28):
        for j in range(0, 280, 28):
            im = imgs[index]
            im = Image.fromarray(im, mode='L')
            new_im.paste(im, (i ,j))
            index += 1

    new_im.save(name)


In [2]:
h_dim = 20
batchsz = 512
lr = 1e-3

In [3]:
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()
x_train, x_test = x_train.astype(np.float32) / 255., x_test.astype(np.float32) /255

# we do not need label
train_db = tf.data.Dataset.from_tensor_slices(x_train)
train_db = train_db.shuffle(batchsz * 5).batch(batch_size=batchsz)
test_db = tf.data.Dataset.from_tensor_slices(x_test)
test_db = test_db.batch(batch_size=batchsz)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [4]:
class AE(keras.Model):
    def __init__(self):
        super(AE, self).__init__()  # 调用父类的初始化函数

        # Encoders
        self.encoder = tf.keras.Sequential([
            tf.keras.layers.Dense(256, activation=tf.nn.relu),
            tf.keras.layers.Dense(128, activation=tf.nn.relu),
            tf.keras.layers.Dense(h_dim, activation=tf.nn.relu),
        ])

        # Decoders
        self.decoder = tf.keras.Sequential([
            tf.keras.layers.Dense(128, activation=tf.nn.relu),
            tf.keras.layers.Dense(256, activation=tf.nn.relu),
            tf.keras.layers.Dense(784),
        ])

    def call(self, inputs, training=None):
        # [b, 784] => [b, 10]
        h = self.encoder(inputs)
        # [b, 10] => [b, 784]
        x_hat = self.decoder(h)

        return x_hat

model = AE()
model.build(input_shape=(None, 784))
model.summary()


Model: "ae"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (None, 20)                236436    
_________________________________________________________________
sequential_1 (Sequential)    (None, 784)               237200    
Total params: 473,636
Trainable params: 473,636
Non-trainable params: 0
_________________________________________________________________


In [5]:
optimizer = tf.optimizers.Adam(lr=lr)

In [6]:
for epoch in range(100):
    for step, x in enumerate(train_db):

        # [b, 28, 28] => [b, 784]
        x = tf.reshape(x, [-1, 784])

        with tf.GradientTape() as tape:
            x_rec_logists = model(x)

            rec_loss = tf.losses.binary_crossentropy(x, x_rec_logists, from_logits=True)
            rec_loss = tf.reduce_mean(rec_loss)


        grads = tape.gradient(rec_loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        if step % 100 ==0:
            print(epoch, step, float(rec_loss))

        # evalutation and save
        x = next(iter(test_db))
        logits = model(tf.reshape(x, [-1, 784]))
        x_hat = tf.sigmoid(logits)

        # x [b, 784] => [b, 28, 28]
        x_hat = tf.reshape(x_hat, [-1, 28, 28])

        # [b, 28, 28] =>  [2b, 28, 28]
        x_concat = tf.concat([x, x_hat], axis=0)
        x_concat = x_concat.numpy() * 255.
        x_concat = x_concat.astype(np.uint8)
        save_images(x_concat, 'ae_images/rec_epoch_%d.png'%epoch)

0 0 0.6928378343582153
0 100 0.331125408411026
1 0 0.3161986470222473
1 100 0.30843275785446167
2 0 0.2983492612838745
2 100 0.30463555455207825
3 0 0.3003339469432831
3 100 0.29873165488243103
4 0 0.295669823884964
4 100 0.29646652936935425
5 0 0.28497421741485596
5 100 0.2913709580898285
6 0 0.2867877185344696
6 100 0.2954622209072113
7 0 0.28468888998031616
7 100 0.28305166959762573
8 0 0.29208239912986755
8 100 0.2935892641544342
9 0 0.2821807563304901
9 100 0.28092968463897705
10 0 0.2901744544506073
10 100 0.28266751766204834
11 0 0.2775445878505707
11 100 0.2835020124912262
12 0 0.2774159908294678
12 100 0.28485819697380066
13 0 0.2766736149787903
13 100 0.28215473890304565
14 0 0.27722251415252686
14 100 0.2868023216724396
15 0 0.28230297565460205
15 100 0.2785570025444031
16 0 0.27122610807418823
16 100 0.2805258631706238
17 0 0.27583640813827515
17 100 0.2775891125202179
18 0 0.27586328983306885
18 100 0.28493499755859375
19 0 0.2769325077533722
19 100 0.2782527804374695
20 0

In [9]:
# 打印原始图片
x = next(iter(test_db))
logits = model(tf.reshape(x, [-1, 784]))
x_hat = tf.sigmoid(logits)

# x [b, 784] => [b, 28, 28]
x_hat = tf.reshape(x_hat, [-1, 28, 28])

# [b, 28, 28] =>  [2b, 28, 28]
x_concat = tf.concat([x, x_hat], axis=0)
x_concat = x
x_concat = x_concat.numpy() * 255.
x_concat = x_concat.astype(np.uint8)
save_images(x_concat, 'ae_images/ori.png')